# Practical 8

## Calculating a Polygenic Risk Score for Height

This week, you will calculate a polygenic risk score (PRS) for height for your mystery genome. This PRS will provide insight into the impact that genetics may have had on your mystery individual’s height (i.e. does their DNA suggest they would be taller or shorter than average, if the only factor impacting height was genetics). 

You will consider a set of 5,580 SNPs that are known to have a genetic impact on height. This list was created based on genome wide association study (GWAS) summary statistics that were computed as part of Pan-UK BioBank Project [1]. The Pan-UK BioBank dataset was further curated by restricting to sites included in the 1.2 million SNP capture array used to prepare your mystery genome, and then clumping using PLINK (v1.90) with parameters --clump-p1=1e-5 and --clump-r2=0.05 [2,3] to group SNPs that are in linkage disequlibrium (LD). SNPs overlapping positions in the Major Histocompatibility Complex (MHC) region of the genome were also excluded due to the extreme variation that occurs in this region of the genome. 


### Getting Started

<b>If you haven't already done so, start an interactive session</b>

- Sign in to https://ood.huit.harvard.edu/ 
- Navigate to `Interactive Apps → Jupyter Lab - HEB 115`
- Launch a Jupyter Lab session with the following parameters:
    - Number of hours: 2
    - Number of CPUs: 4
- When the session is ready, click “Connect to Jupyter”

<b>Create a working directory (called "practical_8" from which you will run commands and store any files that you generate</b>

```bash
mkdir practical_8
cd practical_8
```

<b>Copy these practical instructions to your working directory and open them as a Jupyter Notebook</b>

```bash
cp ~/153784/practical_instructions/Practical8.ipynb ./
```

Then navigate to the practical_8 directory on the sidebar and click on Practical8.ipynb to open it as a Jupyter Notebook

### Part 1)  Prepare a positions file 

In today’s analysis, you will be considering a set of 5,580 SNPs that are described in the following metadata file: 

`~/153784/data/reference_data/height_associated_snps_metadata.txt`

This week we will be using bcftools mpileup [4] to examine the DNA sequences that overlap each position of interest. Like samtools mpileup, bcftools mpileup takes a positions list of the same format to specify the genomic positions that it should analyze. Create a position list file for the 5,580 SNPs of interest and save it in your `practical_8` working directory. 


### Part 2) Calculate genotype likelihoods at each position

Run the following command to use bcftools mpileup [4] to compute genotype likelihoods at each position of interest. This will take some time to run, so I reccomend submitting it as a job using `sbatch`.

```bash
bcftools mpileup -f ~/153784/data/reference_genomes/human_g1k_v37.fasta -T {POINTER TO YOUR POSITIONS LIST FILE} -Q 30 -q 30 -a AD {POINTER TO YOUR MYSTERY BAM} > bcftools_mpileup_output.vcf
```

This will create another output file called bcftools_mpileup_output.vcf that contains information about the reads that align to each position of interest.

Be sure to review the `bcftools mpileup` documentation (https://samtools.github.io/bcftools/bcftools.html#mpileup) to help you understand the vcf file you generated.

A few things to keep in mind about your bcftools results:

1. `bcftools` knows what the reference allele is at the position of interest (since you provided a reference sequence), it does not know what alternative allele to expect. In cases where there are no alternative alleles observed, `bcftools mpileup` will return three comma separated genotype likelihoods in the first part of the last column of the vcf file (i.e. to the left of the colon), corresponding to the likelihood of the homozygous reference, heterozygous, and homozygous alternative states. In cases where one or more alternative alleles are observed, bcftools mpileup will return six comma separated genotype likelihoods. The first three correspond to the genotype likelihoods assuming that the alternative allele matches the observed alternative allele. The last three correspond to the genotype likelihoods assuming that the alternative allele matches something other than what was observed.
2. The genotype likelihood scores are “Phred-scaled,” meaning that the raw likelihood of that genotype is equal to 10-P/10, where P is the Phred score (so the higher the number, the less likely it is that your sample has that genotype). The sum of likelihoods is not necessarily 1.

### Part 3) Use the genotype likelihood calls to calculate genotype dosage

Using the genotype likelihood calls generated for each position of interest, you will need to calculate the genotype dosage. The genotype dosage represents the predicted number of non-reference alleles that your individual has at a particular position, given the data available. It is calculated as follows: 

**Dosage = Pr(Het|Data) + 2*Pr(Alt|Data)**

Tip - I wrote a script (`~/153784/tools/calc_geno_dosage.sh`) to help you convert your Phred-scaled genotype likelihoods into normalized genotype likelihoods (which sum to 1), and then to calculate genotype dosages and report the results in a csv file. To use it you can run the following:


```bash
bash ~/153784/tools/calc_geno_dosage.sh {POINTER TO INPUT VCF FILE} {POINTER TO OUTPUT FILE}
```

### Part 4) Calculate your mystery individual’s PRS 

Now it is time to calculate a PRS for your mystery genome using the following formula: 

$$
PRS_{Height} = \sum_{i=1}^{m}d_{i}\beta_{i}
$$

In this equation, m is the number of SNPs of interest that had a coverage of at least one (i.e. at least one overlapping read), di is the dosage at the ith SNP, and βi is the GWAS-estimated effect size at the ith SNP, which is available in the `height_associated_snps_metadata.txt` file. 

Use the above equation and the dosage information you generated in part 3 to compute a PRS for height for your mystery genome.

*Note* - This equation ignores sites for which there is missing data. This means the PRS value you calculate only considers the SNPs that you have genotype likelihood calls for. 


### Part 5) Prepare an annotated histogram comparing your mystery genome’s score to a 1000 genomes population of your choosing

Finally, you will create an annotated histogram that shows how your mystery individual’s PRS for height compares to other individuals with (somewhat) similar ancestry. 

The following file contains PRSs for 473 individuals from five populations from the 1000 genomes project dataset. 
`~/153784/data/reference_data/1kg_scores.csv`

Select the 1000 genomes population that you think your mystery genome has the most similar ancestry to and plot the distribution of PRSs assigned to each individual in that population in a histogram. Then add a vertical line representing your mystery genome’s computed PRS to show how they compare.

## When you are finished

### Be sure to include the following in your report: 
<b>Methods section</b>: <br>
Be sure to describe each of the methods that you used to calculate genotype likelihoods and dosage at each position of interest and how you used that information to calculate your mystery individual's PRS. Be sure provide an appropriate reference indicating where the list of SNPs of interest and their associated effect sizes came from (i.e. the Pan-UKB dataset [1]) and mention how many SNPs were included in this list.

<b>Results section</b>: <br>
Be sure to include the following in your results section:
- The polygenic risk score that you computed for your mystery genome.
- A histogram demonstrating how your mystery genome’s PRS compares to other individuals from the 1000 genomes population with the most similar ancestry.

<b>Conclusion section</b>: <br>
Based on the results of your analyses:
- Briefly interpret what your mystery genome’s assigned PRS tells you about their height. Do you expect they would be taller than average, shorter than average, or about average based on this PRS if genetics were the only factor controlling height? Be sure to explain how confident you are in this interpretation (and why). 


### Additional Questions to answer at the end of your report: 
1. You may notice that the distribution of PRSs observed among the individuals in the 1000 genomes population that you plotted does not center on zero. Provide a possible explanation for why this might be the case. 
2. In this analysis we used the genotype likelihood scores computed by bcftools mpileup to calculate genotype dosage (i.e. the number of alternative alleles at each position).
    - Why might this not be an ideal approach for low coverage, ancient DNA data.
    - Describe two alternative approaches you could use to calculate dosage. For each, explain whether that approach would be expected to be more or less accurate (or the same) as the approach we took.  
3. Take a look at the script that I provided for calculating genotype dosage (`calc_geno_dosage.sh`). In the section labeled “Calculate dosage,” I used one grep statement followed by five awk statements to calculate dosage and save the results in a temporary file called temp2. Briefly describe what each of the grep and awk statements are doing in this section of the script. (Your answer should be in the form of a numbered list, when you describe each part of the command separately.)


## References
1. Pan-UKB team. https://pan.ukbb.broadinstitute.org. 2020.
2. Chang, Christopher C., et al. "Second-generation PLINK: rising to the challenge of larger and richer datasets." Gigascience 4.1 (2015): s13742-015.
3. www.cog-genomics.org/plink/1.9/ 
4. Danecek, Petr, et al. "Twelve years of SAMtools and BCFtools." Gigascience 10.2 (2021): giab008. https://doi.org/10.1093/gigascience/giab008 
